In [69]:
%load_ext lab_black

# Extract info about funds regisered in KNF 

In [50]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [6]:
path = 'https://www.knf.gov.pl/podmioty/Podmioty_rynku_kapitalowego/Fundusze_Inwestycyjne/TFI_i_FI'
html_doc = requests.get(path)

In [9]:
soup = BeautifulSoup(html_doc.text)

In [18]:
summary_table = soup.find_all("table", {"class": "sample"})[0]

In [42]:
links_regex = re.compile('^#.*$')
country_id_regex = re.compile('Identyfikator krajowy: (PLTFI.{6})')

In [25]:
links_to_sections = summary_table.find_all("a", {'href' : links_regex})

In [31]:
# section in html doc + full name of fund connected with it
sections = list(map(lambda link: link['href'], links_to_sections))
funds_names = list(map(lambda link: link.text, links_to_sections))


In [61]:
headers = []
country_ids = []
dfs = []

for section in sections:
    link_before_section = soup.find('a', {'name' : section[1:]})

    headers.append(link_before_section.findNext('h4').text) 
    country_ids.append(link_before_section.findNext(text=country_id_regex))
    html_df_wrappers = link_before_section.findNext('div', {'class' : 'table-responsive'})
    html_table = html_df_wrappers.find('table')
    dfs.append(pd.read_html(str(html_table))[0])
    # pd.read_html(html_df)


In [65]:
dfs[0]

,fundusz,identyfikator krajowy,data decyzji KNF,data rejestracji w sądzie,nr w rejestrze funduszy inwestycyjnych
0,ABELIA CAPITAL Fundusz Inwestycyjny Zamknięty ...,PLFIZ000030,NaN,17-09-2015,1212
1,AD ROYAL Fundusz Inwestycyjny Zamknięty Aktywó...,PLFIZ000572,08-04-2008,15-10-2008,420
2,AGIO Nieruchomości Komercyjnych Fundusz Inwest...,PLFIZ000037,NaN,24-06-2014,986
3,AGIO PLUS Fundusz Inwestycyjny Otwarty,PLFIO000281,16-07-2015,09-02-2016,1309
4,Wydzielone subfundusze:,Wydzielone subfundusze:,16-07-2015,09-02-2016,1309
...,...,...,...,...,...
67,VALUE Fundusz Inwestycyjny Zamknięty,PLFIZ000779,NaN,06-04-2016,1339
68,Wydzielone subfundusze:,Wydzielone subfundusze:,NaN,06-04-2016,1339
69,Subfundusz 1,PLFIZ000023,NaN,06-04-2016,1339
70,Varsovia Niestandaryzowany Sekurytyzacyjny Fun...,PLFIZ000781,NaN,16-09-2013,895


In [195]:
def restructure_funds_df(df: pd.DataFrame) -> pd.DataFrame:

    fund_regex = re.compile(" (fundusz)(e|y|)", re.IGNORECASE)
    subfind_start_regex = re.compile(".*wydzielone.subfundusze.*", re.IGNORECASE)

    new_df = pd.DataFrame(
        columns=["fundusz", "subfundusz", "indentyfikator_krajowy", "nr_w_rejestrze"]
    )

    is_subfund = False
    curr_fund = ""

    for index, row in df.iterrows():

        if subfind_start_regex.search(row["fundusz"]):
            is_subfund = True
            continue

        if not is_subfund:
            curr_fund = row["fundusz"]

        if is_subfund:
            if bool(fund_regex.search(row["fundusz"])):
                is_subfund = False

        new_row = pd.DataFrame()

        if is_subfund:
            new_row = pd.DataFrame(
                {
                    "fundusz": [curr_fund],
                    "subfundusz": [row["fundusz"]],
                    "indentyfikator_krajowy": [row["identyfikator krajowy"]],
                    "nr_w_rejestrze": [row["nr w rejestrze funduszy inwestycyjnych"]],
                }
            )
        else:
            new_row = pd.DataFrame(
                {
                    "fundusz": [row["fundusz"]],
                    "subfundusz": [None],
                    "indentyfikator_krajowy": [row["identyfikator krajowy"]],
                    "nr_w_rejestrze": [row["nr w rejestrze funduszy inwestycyjnych"]],
                }
            )
        new_df = new_df.append(new_row, ignore_index=True)

    return new_df

In [196]:
df_tmp = restructure_funds_df(dfs[0])

C:\Users\Dawid\AppData\Local\Temp\ipykernel_22100\3538954451.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)


In [197]:
df_tmp

,fundusz,subfundusz,indentyfikator_krajowy,nr_w_rejestrze
0,ABELIA CAPITAL Fundusz Inwestycyjny Zamknięty ...,None,PLFIZ000030,1212
1,AD ROYAL Fundusz Inwestycyjny Zamknięty Aktywó...,None,PLFIZ000572,420
2,AGIO Nieruchomości Komercyjnych Fundusz Inwest...,None,PLFIZ000037,986
3,AGIO PLUS Fundusz Inwestycyjny Otwarty,None,PLFIO000281,1309
4,AGIO PLUS Fundusz Inwestycyjny Otwarty,Subfundusz AGIO Akcji PLUS,PLFIO000264,1309
...,...,...,...,...
63,TOROX EKO Fundusz Inwestycyjny Zamknięty Aktyw...,None,PLFIZ000730,1295
64,VALUE Fundusz Inwestycyjny Zamknięty,None,PLFIZ000779,1339
65,VALUE Fundusz Inwestycyjny Zamknięty,Subfundusz 1,PLFIZ000023,1339
66,Varsovia Niestandaryzowany Sekurytyzacyjny Fun...,None,PLFIZ000781,895


In [198]:
df_tmp.to_csv("./tmp.csv")

In [134]:
dfs[0]

,fundusz,identyfikator krajowy,data decyzji KNF,data rejestracji w sądzie,nr w rejestrze funduszy inwestycyjnych
0,ABELIA CAPITAL Fundusz Inwestycyjny Zamknięty ...,PLFIZ000030,NaN,17-09-2015,1212
1,AD ROYAL Fundusz Inwestycyjny Zamknięty Aktywó...,PLFIZ000572,08-04-2008,15-10-2008,420
2,AGIO Nieruchomości Komercyjnych Fundusz Inwest...,PLFIZ000037,NaN,24-06-2014,986
3,AGIO PLUS Fundusz Inwestycyjny Otwarty,PLFIO000281,16-07-2015,09-02-2016,1309
4,Wydzielone subfundusze:,Wydzielone subfundusze:,16-07-2015,09-02-2016,1309
...,...,...,...,...,...
67,VALUE Fundusz Inwestycyjny Zamknięty,PLFIZ000779,NaN,06-04-2016,1339
68,Wydzielone subfundusze:,Wydzielone subfundusze:,NaN,06-04-2016,1339
69,Subfundusz 1,PLFIZ000023,NaN,06-04-2016,1339
70,Varsovia Niestandaryzowany Sekurytyzacyjny Fun...,PLFIZ000781,NaN,16-09-2013,895


In [85]:
dfs[0].iloc[-1]["fundusz"]

'VIDRIO FUND Fundusz Inwestycyjny Zamknięty Aktywów Niepublicznych'

In [200]:
full_df = pd.DataFrame()
for df in dfs:
    full_df = pd.concat([full_df, restructure_funds_df(df)])

C:\Users\Dawid\AppData\Local\Temp\ipykernel_22100\3538954451.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)
C:\Users\Dawid\AppData\Local\Temp\ipykernel_22100\3538954451.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)
C:\Users\Dawid\AppData\Local\Temp\ipykernel_22100\3538954451.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)
C:\Users\Dawid\AppData\Local\Temp\ipykernel_22100\3538954451.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ign

In [202]:
full_df.reset_index(drop=True)

,fundusz,subfundusz,indentyfikator_krajowy,nr_w_rejestrze
0,ABELIA CAPITAL Fundusz Inwestycyjny Zamknięty ...,None,PLFIZ000030,1212
1,AD ROYAL Fundusz Inwestycyjny Zamknięty Aktywó...,None,PLFIZ000572,420
2,AGIO Nieruchomości Komercyjnych Fundusz Inwest...,None,PLFIZ000037,986
3,AGIO PLUS Fundusz Inwestycyjny Otwarty,None,PLFIO000281,1309
4,AGIO PLUS Fundusz Inwestycyjny Otwarty,Subfundusz AGIO Akcji PLUS,PLFIO000264,1309
...,...,...,...,...
1389,Universe 2 Niestandaryzowany Sekurytyzacyjny F...,None,PLFIZ000774,1079
1390,Universe 3 Niestandaryzowany Sekurytyzacyjny F...,None,PLFIZ000775,1078
1391,Universe Niestandaryzowany Sekurytyzacyjny Fun...,None,PLFIZ000777,1028
1392,Value Fund Poland Activist Fundusz Inwestycyjn...,None,PLFIZ000809,1421
